# Ramayana RAG System 🏹

This notebook implements a **Retrieval-Augmented Generation (RAG)** system using the ancient Indian epic **Ramayana** as the knowledge base.

## How to Use:
1. Run all cells in order (Ctrl+Shift+Enter for each cell)
2. Once setup is complete, use: `ask_ramayana_question("your question here")`
3. Test with sample questions about Rama, Sita, Hanuman, and other characters

## Features:
- 🔍 Semantic search through the Ramayana text
- 🤖 AI-powered responses using Azure OpenAI
- 📚 Comprehensive coverage of the epic
- ⚡ Fast vector-based retrieval

---

In [61]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter, SentenceTransformersTokenTextSplitter 
from langchain_community.document_loaders import TextLoader
import json
import numpy as np

# Load environment variables
load_dotenv()

True

In [62]:
# Initialize Azure OpenAI models with error handling
print("Initializing Azure OpenAI models...")

# Try to find working embedding deployment
embedding_deployments_to_try = ["text-embedding-3-large", "text-embedding-ada-002", "text-embedding-3-small"]

embedding_model = None
for deployment_name in embedding_deployments_to_try:
    try:
        test_embedding_model = AzureOpenAIEmbeddings(
            azure_deployment=deployment_name,
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_VERSION")
        )
        # Test if the deployment works
        test_embed = test_embedding_model.embed_query("test")
        print(f"✓ Found working embedding deployment: '{deployment_name}'")
        embedding_model = test_embedding_model
        break
    except Exception as e:
        print(f"✗ '{deployment_name}' failed: {e}")
        continue

if embedding_model is None:
    raise Exception("No working embedding deployment found. Please check your Azure OpenAI configuration.")

# Initialize chat model  
chat_model = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_VERSION")
)

print("✓ Azure OpenAI models initialized successfully!")

Initializing Azure OpenAI models...
✓ Found working embedding deployment: 'text-embedding-3-large'
✓ Azure OpenAI models initialized successfully!


In [63]:
# Set up file paths
current_dir = os.getcwd()
file_path = os.path.join(current_dir, "input", "ramayana.txt")
db_dir = os.path.join(current_dir, "ramayana_db")

# Check if the Ramayana text file exists
if not os.path.exists(file_path):
    raise FileNotFoundError(f"The file {file_path} does not exist. Please make sure you have the ramayana.txt file in the input folder.")

print(f"✓ Found Ramayana text file: {file_path}")

# Load and process the document
print("Loading Ramayana text...")
loader = TextLoader(file_path, encoding="utf-8")
documents = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200  # Some overlap to maintain context
)

docs = text_splitter.split_documents(documents)

print(f"✓ Document loaded and split into {len(docs)} chunks")
print(f"Sample chunk: {docs[0].page_content[:200]}...")

✓ Found Ramayana text file: c:\Users\285104\Downloads\Langchain-RAG\Langchain-RAG\input\ramayana.txt
Loading Ramayana text...
✓ Document loaded and split into 37 chunks
Sample chunk: The Ramayana - Epic of Sri Rama

This is a comprehensive collection of the Ramayana, one of the greatest epics of ancient India, composed by the sage Valmiki. The Ramay...


In [64]:
# Create or load vector store
print("Setting up vector store...")

persistent_directory = os.path.join(db_dir, "ramayana_faiss_db")

if not os.path.exists(persistent_directory):
    print("Creating new vector store...")
    os.makedirs(db_dir, exist_ok=True)
    
    # Create FAISS vector store with embeddings
    vector_store = FAISS.from_documents(docs, embedding_model)
    
    # Save the vector store
    vector_store.save_local(persistent_directory)
    print(f"✓ Vector store created and saved to {persistent_directory}")
else:
    print("Loading existing vector store...")
    vector_store = FAISS.load_local(
        persistent_directory, 
        embedding_model, 
        allow_dangerous_deserialization=True
    )
    print("✓ Vector store loaded successfully")

# Create retriever
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Return top 3 most relevant chunks
)

print("✓ Retriever configured successfully")

Setting up vector store...
Loading existing vector store...
✓ Vector store loaded successfully
✓ Retriever configured successfully


In [69]:
# Test retrieval functionality
test_query = "Who is Hanuman and what are his powers?"
print(f"Testing retrieval with query: '{test_query}'")

relevant_docs = retriever.invoke(test_query)
print(f"✓ Retrieved {len(relevant_docs)} relevant documents")

for i, doc in enumerate(relevant_docs[:2]):  # Show first 2 results
    print(f"\nDocument {i+1} excerpt:")
    print(doc.page_content[:300] + "...")

Testing retrieval with query: 'Who is Hanuman and what are his powers?'


AssertionError: 

In [71]:
def ask_ramayana_question(question):
    """
    Ask a question about the Ramayana and get an AI-powered answer
    
    Args:
        question (str): Your question about the Ramayana
        
    Returns:
        str: AI-generated answer based on relevant text chunks
    """
    try:
        # Retrieve relevant documents
        relevant_docs = retriever.invoke(question)
        
        # Prepare context from retrieved documents
        context = "\n\n".join([doc.page_content for doc in relevant_docs])
        
        # Create the prompt
        prompt = f"""Based on the following excerpts from the Ramayana epic, please answer the question.

Question: {question}

Relevant Content from Ramayana:
{context}

Please provide a detailed and accurate answer based only on the provided content. If the answer cannot be found in the content, say "I don't have enough information to answer this question based on the provided content."
"""
        
        # Get response from chat model
        messages = [
            SystemMessage(content="You are a knowledgeable assistant specializing in the Ramayana epic. Provide accurate and detailed answers based on the given content."),
            HumanMessage(content=prompt)
        ]
        
        response = chat_model.invoke(messages)
        return response.content
        
    except Exception as e:
        return f"Error occurred: {str(e)}"

print("✓ RAG question-answering function ready!")

✓ RAG question-answering function ready!


In [72]:
# Test the RAG system with sample questions
print("🧪 Testing Ramayana RAG System")
print("="*50)

sample_questions = [
    "Who is Hanuman and what are his special powers?",
    "Why was Rama exiled from Ayodhya?",
    "How did Rama and Sita first meet?",
    "What happened during the battle between Rama and Ravana?",
    "Who helped Rama build the bridge to Lanka?",
    "What are the main teachings of the Ramayana?"
]

for i, question in enumerate(sample_questions[:3], 1):  # Test first 3 questions
    print(f"\n📝 Question {i}: {question}")
    print("-" * 50)
    
    answer = ask_ramayana_question(question)
    print(f"💬 Answer: {answer}")
    print("=" * 50)

print("\n🎉 RAG system testing complete!")
print("✓ You can now use ask_ramayana_question('your question') to ask about the Ramayana")

🧪 Testing Ramayana RAG System

📝 Question 1: Who is Hanuman and what are his special powers?
--------------------------------------------------
💬 Answer: Error occurred: 

📝 Question 2: Why was Rama exiled from Ayodhya?
--------------------------------------------------
💬 Answer: Error occurred: 

📝 Question 3: How did Rama and Sita first meet?
--------------------------------------------------
💬 Answer: Error occurred: 

🎉 RAG system testing complete!
✓ You can now use ask_ramayana_question('your question') to ask about the Ramayana


In [ ]:
# Interactive interface for asking questions
print("🎯 Interactive Ramayana Q&A Interface")
print("="*40)
print("Your Ramayana RAG system is ready!")
print()
print("📖 Sample questions you can ask:")
print("• 'What powers does Hanuman have?'")
print("• 'How did Sita prove her purity?'")
print("• 'Why did Vibhishana join Rama?'")
print("• 'What happened to Ravana in the end?'")
print("• 'How was the bridge to Lanka built?'")
print()
print("💡 Usage examples:")
print("   answer = ask_ramayana_question('Who is the main character of Ramayana?')")
print("   print(answer)")
print()
print("🚀 Ready to answer your questions about the Ramayana!")